#   Identification and validation of weak ties

This notebook contains the code for the implementation of the graphical analysis inorder to identify the weak ties and also to validate the results.

### Background Theory
There are several algorithms of link prediction algorithms based on node, topology and social thoery: that can be used in the MYMK (Members You May Know) recommendation system. This work uses the concepts of social theory to determine the tie strength between two users. There are seven dimensions that has to be considered to accurately predict the tie strength they are : Intensity of interaction, Intimacy, reccency of interaction, reciprocity, emmotional suppport, structure of connections and social distance. The actual tie strength between two users in real life cannot be exactly modeled by the data that is available on the Xing platform. Thus we use topology and socail theory concepts to identify the weak ties.

### Ground truths
> There cannot exists an quantitative value between two users that represent the strength of the mutual connection between the users similar to that of a real world relation. However we can have a binary distinction between a strong and a weak tie in the network

> Strong triadic closure is the property among three nodes A, B, and C, such that if a strong tie exists between A-B and A-C, there is a weak or strong tie between B-C

> Local bridge is an edge that connects two subgraphs. This Local bridge is always a weak tie.

### Hypothesis
refer the video below:

Let "A" be the user to who contact recommendations are to be made. "A" is connected to "B" and "B" is connected to "E".This implies that "E" is a second degree contact or "A". 

According to hypothesis, recommending "B" a connection with "E" is a bad idea if the connection between "A" & "B"  or between "B" and "E" is a local bridge (weak tie) and if there are no mutual friends between "A" and "E" apart from "B".

Therefore, we have 3 conditions that needs to be implemented:
> Condition 1 : E is a second degree contact for A.

> Condition 2 : Either one of the connection between A and B OR between B and E should be a local bridge.

> Condition 3 : There are no other mutual friends between A and E apart from B.

Outcome after the application of all the above conditions: We get a list of connections (edges) that according to the hypothesis is NOT a good recommendation as they have very low likelihood of knowing each other.

In [1]:
from IPython.display import HTML
HTML("""
<video width="640" height="480" controls>
  <source src="hypothesis.mp4" type="video/mp4">
</video>
""")

NetworkX is a python library that is used to analyze the graphs. It can import a txt file that contains a list of pair of nodes that form an edge. The library itself checks for duplication of connections. It is necessary to keep in mind if it a directeed or undirected graph before importing. In this case it is a undirected, unweighed, one dimensional graph.

In [4]:
import random
import numpy as np
from importlib import reload
import matplotlib.pyplot as plt
import networkx as nx

# real dataset
data_set = nx.read_edgelist('data/sample-ch2017.txt')
no_node = nx.number_of_nodes(data_set)
print (nx.info(data_set))

Name: 
Type: Graph
Number of nodes: 31135
Number of edges: 642287
Average degree:  41.2582


### Total connections
The formula used to find the total number of edges that can be created is ggiven by the formula f(n) = (n$*$(n-1))$/$2 
This incldes both existing links and also the ones that can be created.

In [5]:
# number of edges if the graph was complete : f(n) = (n*(n-1))/2
#cg = nx.complete_graph(no_node) # takes too long to execute thus hashed
#print(nx.number_of_nodes(cg))

no_edges_possible = (no_node * (no_node - 1)) / 2
print("Number of edges if the graph was complete :", no_edges_possible) 

Number of edges if the graph was complete : 484678545.0


### Local Bridges
Local bridges are those connections that act as an connection between two communities that have some factor in common. Following snippet of code identifies all the local bridges in the network and the nodes that are involved in the formation of local bridges.

In [6]:
# Local bridges (lb) (no common contacts)
lb = list(nx.local_bridges(data_set , with_span = False))
print('Number of Local bridges in 2017: ', len(list(nx.local_bridges(data_set, with_span = False))))

# To create a list of nodes that belong to local node. from [('a','b'), ('c','b')] to ['a','b','c','b']
new_lb = []
for tup in lb:
    for element in tup:
        new_lb.append(element)  
        
#nd is to remove duplicates
nodes_in_lb = list(set(new_lb))

print('Number of nodes included in a local bridges in 2017:', len(nodes_in_lb))

Number of Local bridges in 2017:  96038
Number of nodes included in a local bridges in 2017: 30013


### Condition 1: Finding all the second degree connections (sdc)

In [7]:
# to find second degree contacts of a single node
def find_nodes(graph, node, distance):
     # get all nodes within distance around the query node
     nodes = set(nx.ego_graph(graph, node, radius=distance))
     # remove nodes that are not **at** the specified distance but closer: removes all the first degree contact
     if distance > 1:
      nodes -= set(nx.ego_graph(graph, node, radius=distance-1))
     return list(nodes)

# example implementation
#ba = find_nodes(data_set, nodes_in_lb[4], 2) # reeturns a list of all the nodes tat are 2nd deg
#print((len(ba)))
#ba


### Condition 2: Identifying all the sdc tha pass through a node that is a part of local bridge

In [8]:
# second degree contacts that pass through the LB of a single node
# two parameters that it takes are 1) the graph itself here data_set 2) one specific local bridge (pair of nodes)
# it returns : predictive edges between the [second degree node, the node that is the part of LB]  in that same order
# returned variable is a nested list with the order 2
def seconddegree (data_set, node_pair):
    b = node_pair[0]
    sdc_through_lb = []
    n = list(data_set.neighbors(node_pair[1]))
    pair =[]
    
    for a in range(len(n)):
        lis = []
        lis.append(n[a])
        lis.append(b)
        sdc_through_lb.append(lis)
    return sdc_through_lb

# example implementation
#vr = seconddegree(data_set, lb[0])
#print(vr)

In [9]:
# works !! to get all the second degree edges with their respective nodes ! 
# It applies the above mentioned function to all the pair of nodes that form a local bridge
# returns nested list of order 3 . 
# non existing edges ! predicted false edges
# second node in this return set (pe) is always the node that is the part of a local bridge
def allnodesseconddegree (data_set, lb):
    sdc = []
    for a in range(len(lb)):
        var = seconddegree(data_set, lb[a])
        sdc.append(var)
    return sdc

lb_second_degree_nodes = allnodesseconddegree(data_set, lb)
#lb_second_degree_nodes

### Condition 3: Identify the edges that have no mutual friend apart fromm the one that is a part of local bridge formation. Returns the edges that satisfy the hypothesis.

In [11]:
#now for the last condition ! No other mutual friend apart from ***
# returns the nested list of all the edges(node pair) that have only one mutual friend that is a part of LB
interseclist = []
satisfy_hypothesis = []

for a in range(len(lb_second_degree_nodes)):
    for b in range(len(lb_second_degree_nodes[a])):
        cc = lb_second_degree_nodes[a]
        
        for c in range(len(cc[b])):
            dd = cc[b]
            
            if dd[0] != dd[1]:
                n1 = set(list(data_set.neighbors(dd[0])))
                n2 = set(list(data_set.neighbors(dd[1])))
                intersection = list(n1.intersection(n2))
                ls = []
                
                if len(intersection) == 1 :
                    ls.append(dd[0])
                    ls.append(dd[1])
                    satisfy_hypothesis.append(ls)

print("Number of edges that satisfy the hypothesis: (needs to be cleaned for duplications, reverse sequence of nodes)", len(satisfy_hypothesis))

Number of edges that satisfy the hypothesis: (needs to be cleaned for duplications, reverse sequence of nodes) 7086940


### Post processing clean up!
The resulting list of edges can contain some duplications and reversed lists.

In [12]:
# remove exact replications : eg [a,b] & [a,b] then only one is kept 
sh_tuples = set(map(tuple, satisfy_hypothesis))  #need to convert the inner lists to tuples so they are hashable
no_duplication = list(map(list, sh_tuples))
print("cleaned duplications:",len(no_duplication))

# works : if . [a,b] and [b, a] are in the list then only one either [a,b] or [b,a] is selected
temp = set(frozenset(x) for x in no_duplication)
lst = [list(x) for x in temp]
#print(len(lst))

#to remove duplications: [a,a] should be discarded
all_hypothesis_edges = []
for a in range(len(lst)):
    temp1 = lst[a]
    list_holder = []
    if temp1[0] != temp1[1]:
        list_holder.append(temp1[0])
        list_holder.append(temp1[1])
        all_hypothesis_edges.append(list_holder)
print("(cleaned for reversed nodes) Final number of edges satisfying the hypothesis",len(all_hypothesis_edges))

# all_hypothesis_edges is the final list to use ! all_hypothesis_edges give a list of edges that are predicted to not get connected

cleaned duplications: 3543470
(cleaned for reversed nodes) Final number of edges satisfying the hypothesis 3343196


## Offline Evaluation

#### Training and Testing set : 
The network as of year 2017 is the training set and network as of year 2018 is the testing set.

In [13]:
# import 2018 dataset
data_set2 = nx.read_edgelist('data/sample-ch2018.txt')
node = nx.number_of_nodes(data_set2)
print (nx.info(data_set2))
fn = (node * (node - 1)) / 2
print('Number of edges that can be established:', fn)

Name: 
Type: Graph
Number of nodes: 26276
Number of edges: 672589
Average degree:  51.1942
Number of edges that can be established: 345200950.0


### Validation


To find the overlap of ties between the two sets 2017 and 2018 the list of edges can have a reversed directions as well. Thus we check for the overlap in both the directions and sum theem up to get the final number.

In [14]:
# to check the extra edges that was created in 2018 and did not exist in 2017
edg2017 = list(data_set.edges)
edges2018 = list(data_set2.edges)

direct_intesection = list(set(edg2017) & set(edges2018)) # intersection between the two
#print(len(direct_intesection))

# reverse checks
#print(edg2017[7])

# to convert all the tuples to list so that they can then be reversed
l = []
for a in range(len(edg2017)):
    ls = edg2017[a]
    l.append(list(ls))

# reversing the edg2017 list
for a in range(len(l)):
    l[a].reverse()

# reverse checks
#print("Reversed list:", l[7])

#converting the list back to tuples so that the intersectio can be found
lis =[]
for a in range(len(l)):
    ls = l[a]
    lis.append(tuple(ls))
    
#print("Reversed list:",  lis[7])    
   
reversed_intersection = list(set(lis).intersection(set(edges2018)))
#print(len(reversed_intersection))

#intesecction of 2017 and 2018 
total_intersection = reversed_intersection + direct_intesection
print('Edges intersection length is:', len(total_intersection))

Edges intersection length is: 622635


In [15]:
# difference
difference = list((set(edges2018)).difference(set(total_intersection)))
print("Intersection between 2017 and 2018 dataset:",len(difference))


Intersection between 2017 and 2018 dataset: 49954


In the following snippet we identify the edges that overlap with the list of links that satisfy the hypothesis. The result of this is the number of false positives of the hpothesis.

In [16]:
# cleaning up data before comparing with 2018 dataset

edges = list(data_set2.edges)

# straight nodes to match them all
into_tuples = []
for a in range(len(all_hypothesis_edges)):
    ls = all_hypothesis_edges[a]
    into_tuples.append(tuple(ls))
    

# straight list overrlap
#using sets to solve time consuming stuff
direct_overlap = list(set(into_tuples) & set(edges))
#print(len(direct_overlap))
#print(direct_overlap)


# reversing the nodes to match them all
# this reverse command operates on the list itself :: inplace reverse operation!!

#print((all_hypothesis_edges[7])) # before reversing the list
for a in range(len(all_hypothesis_edges)):
    all_hypothesis_edges[a].reverse()
#print((all_hypothesis_edges[7]))

rev = []
for a in range(len(all_hypothesis_edges)):
    ls = all_hypothesis_edges[a]
    rev.append(tuple(ls))


# reversed list of lsst overlap
#using sets to solve time consuming stuff
reversed_overlap = list(set(rev) & set(edges))
#print(len(reversed_overlap))
#print(reversed_overlap)

total_edges_overlap = direct_overlap + reversed_overlap
print("Number of edges that satisfy hypothesis and were alsoo fouund in 2018 dataset: ",len(total_edges_overlap))
# exports
np.savetxt("edges_that_overlap.csv", total_edges_overlap, delimiter=",", fmt='%s')

Number of edges that satisfy hypothesis and were alsoo fouund in 2018 dataset:  1038


Thus the reslut, 1.038 links out of 49.954 created were the false positives.

## Post analysis

After carefull considerations, any of the following three couls be a cause of connection.

> customers lifetime on the Xing platform may corelate with the 1038 connections:: reason -- The strong ties itself may not be established yet so below a threshold number of contact this hypothesis may not be useful.

> May have some similar attributes !! same school or university or something like that would form a cluster but an influential relation will not cause a cluster formation thus these connections could be one way relation as well.

> Network is cut off during sampling !! There may be some other friends that are not included in the sample this will influence the result.


The impact of sampling on the results can be found by checking if there are any other mutual friends. Thus the following code snippet.

In [19]:
# All the fiends from Xing database of the nodes that are a part of edges that satisfy the hypothesis (1.038)
all_friends = nx.read_edgelist('data/all_friends_of_users_involved_in_1038.csv',  delimiter=',', nodetype=int, encoding="utf-8")

# edge list that satisfy the hypothesis
hyp_edges = nx.read_edgelist('edges_that_overlap.csv', delimiter=',', nodetype=int, encoding="utf-8")

# 2018 dataset
set2018 = nx.read_edgelist('data/sample-ch2018.txt', nodetype=int, encoding="utf-8")

# 2017 dataset
set2017 = nx.read_edgelist('data/sample-ch2017.txt', nodetype=int, encoding="utf-8")

ed = list(hyp_edges.edges)

li =[]

for a in range (len(ed)):
    aa = ed[a]
    for b in range (len(aa)):
        x = aa[0]
        y = aa[1]
        
        set2018mx = set(set2018.neighbors(x))
        set2018my = set(set2018.neighbors(y))
        
        afnx = set(all_friends.neighbors(x))
        afny = set(all_friends.neighbors(y))
        
        set2017px = set(set2017.neighbors(x))
        set2017py = set(set2017.neighbors(y))
        
        m = set2018mx & set2018my
        n = afnx & afny
        p = set2017px & set2017py
        
    l=[]
    if len(p) == len(n):
        l.append(x)
        l.append(y)
        li.append(l)

print("length of the list that does not have any other mutual friends from the whole data set of friends: ",len(li))

length of the list that does not have any other mutual friends from the whole data set of friends:  101
